In [20]:
import requests
from bs4 import BeautifulSoup
import pandas
import argparse
import connect

ModuleNotFoundError: No module named 'connect'

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--page_num_max", help = "Enter the number of pages to parse", type="int")
parser.add_argument("--dbname", help="Enter the name of db", type="str")
args = parser.parse_args()

In [12]:
oyo_url = "https://www.oyorooms.com/hotels-in-bangalore/?page="

In [13]:
page_num = 3

In [14]:
scraped_info_list = []

In [ ]:
connect.connect(args.dbname)

In [18]:
for i in range(1, page_num):
    url = oyo_url + str(i)
    print("get Request for: " + url)
    
    req = requests.get(url)
    content = req.content
    soup = BeautifulSoup(content, "html.parser")
    
    all_hotels = soup.find_all("div", {"class" : "hotelCardListing"})    

    for hotel in all_hotels:
        hotels = {}
        hotels["name"] = hotel.find("h3", {"class": "listingHotelDescription__hotelName"}).text
        hotels["address"] = hotel.find("span", {"itemprop": "streetAddress"}).text
        hotels["price"] = hotel.find("span", {"class": "listingPrice__finalPrice"}).text  

        try:
            hotels["rating"] = hotel.find("span", {"class": "hotelRating__ratingSummary"}).text
        except AttributeError:
            pass

        parent_amenities_element = hotel.find("div", {"class: amenityWrapper"})

        amenities_list = []
        for amenity in parent_amenities_element.find_all("div", {"class: amenityWrapper__amenity"}):
            amenities_list.append(amenity.find("span", {"class":"d-body-sm"}).text.strip())

        hotels["amennties"] = ', '.join(amenities_list[:-1])
        scraped_info_list.append(hotels)
        connect.insert_into_table(args.dbname, tuple(hotels.values()))

In [19]:
dataFrame = pandas.DataFrame(scraped_info_list)
print("Creating csv file ")
dataFrame.to_csv("Oyo.csv")
connect.get_hotel_info(args.dbname)